# Model Project: The general Solow model with land
**Introduction to Programming and Numerical Analysis**

In this project, the general Solow model with land is solved analytically and simulated. Then, the model is extended with finite natural resources.

Imports and set magics:

In [ ]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
# import modelproject

**Table of contents**<a id='toc0_'></a>    
- 1. [Model](#toc1_)    
- 2. [Analytical solution](#toc2_)    
- 3. [Numerical solution](#toc3_)    
- 4. [Further analysis](#toc4_)    
- 5. [Model extension](#toc5_)
- 6. [Conclusion](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Model](#toc0_)

The **Solow growth model** is a macroeconomic model constructed to explain the long-run growth of an economy through dynamics concerning population growth, capital accumulation and technological innovation. 

The **general Solow growth model with land** includes land as a fixed resource. This model abstracts from human capital, which would sligthly complicate the analysis without notably changing the conclusions. It is worth noting that the model does not specify any microfoundations for households. Therefore, parameters such as the savings rate are taken as exogenously given.

The model is set up in **discrete time** rather than continuous time. The notation in this project follows chapter 5 in Sørensen, P. and Whitta-Jacobsen, H. J. (2010), *Introducing Advanced Macroeconomics*.

The production function is a standard Cobb-Douglas production function with *Harrod neutral* technology defined as
$$
Y_t = K_t^{\alpha}(A_tL_t)^{\beta}X^{\kappa},
$$
where $\alpha, \beta, \kappa > 0$ and $\alpha + \beta + \kappa = 1$. 

$Y_t$ is output in period $t$. <br>
$K_t$ is the capital stock in period $t$. <br>
$A_t$ is the level of technology in period $t$. <br>
$X$ is the fixed amount (area) of land. <br>

The capital stock accumulates by
$$
K_{t+1} = sY_t + (1-\delta)K_t,
$$
where $0<s<1$ is the exogenous savings rate (so savings are $S_t=sY_t$) and $0 < \delta < 1$ is the rate at which capital depreciates. $K_0$ is given.

The labor force evolves as
$$
L_{t+1} = (1+n) L_t,
$$
where $n>-1$ is the exogenous population growth rate. $L_0$ is given.

The level of technology evolves as
$$
A_{t+1} = (1+g) A_t,
$$
where $g$ is the exogenous technology growth rate. $A_0$ is given.

Now, define the capital-output ratio
$$
z_t \equiv \frac{K_t}{Y_t},
$$
which will be useful for solving the model. Substitute for $Y_{t+1}$ in $z_{t+1}$
$$
z_{t+1} = \frac{K_{t+1}}{Y_{t+1}} = \frac{K_{t+1}}{K_{t+1}^{\alpha}(A_{t+1}L_{t+1})^{\beta}X^{\kappa}} = \frac{K_{t+1}^{1-\alpha}}{(A_{t+1}L_{t+1})^{\beta}X^{\kappa}}
$$
Insert for $K_{t+1}$:
$$
z_{t+1} = \frac{(sY_t + (1-\delta)K_t)^{1-\alpha}}{(A_{t+1}L_{t+1})^{\beta}X^{\kappa}} = \frac{(s + (1-\delta)z_t)^{1-\alpha}}{(A_{t+1}L_{t+1})^{\beta}X^{\kappa}}Y_{t}^{1-\alpha}
$$
Substitute for $A_{t+1}$ and $L_{t+1}$:
$$
\begin{align*}
z_{t+1} &= \frac{(s + (1-\delta)z_t)^{1-\alpha}}{((1+g) A_t (1+n) L_t)^{\beta}X^{\kappa}}Y_t^{1-\alpha}\\
&= \frac{(s + (1-\delta)z_t)^{1-\alpha}}{[(1+g)(1+n)]^{\beta}}\frac{Y_t^{1-\alpha}}{(A_{t}L_{t})^{\beta}X^{\kappa}}\\
&=\frac{(s + (1-\delta)z_t)^{1-\alpha}}{[(1+g)(1+n)]^{\beta}}\frac{Y_t^{1-\alpha}}{(A_{t}L_{t})^{\beta}X^{\kappa}}\frac{K_t^{\alpha}}{K_t^{\alpha}}\\
&=\frac{(s + (1-\delta)z_t)^{1-\alpha}}{[(1+g)(1+n)]^{\beta}}Y^{-\alpha}K^{\alpha}\\
&=\frac{(s + (1-\delta)z_t)^{1-\alpha}}{[(1+g)(1+n)]^{\beta}}z_t^{\alpha}\\
\end{align*}
$$
Rearrange as 
$$
\begin{equation}
z_{t+1} =\frac{1}{[(1+g)(1+n)]^{\beta}}[s + (1-\delta)z_t]^{1-\alpha}z_t^{\alpha}\\
\end{equation}
$$
In steady state, $z_{t+1}=z_t=z^*$:
$$
z^* =\frac{1}{[(1+g)(1+n)]^{\beta}}[s + (1-\delta)z^*]^{1-\alpha}\left(z^*\right)^{\alpha}\\
$$

## 2. <a id='toc2_'></a>[Analytical solution](#toc0_)

The package `sympy` is used to do symbolic math and derive the analytical solution to the model. First, the symbols of the model's variables and parameters are defined.

In [ ]:
# define symbols
z = sm.symbols('z')
alpha = sm.symbols('alpha')
beta = sm.symbols('beta')
kappa = sm.symbols('kappa')
s = sm.symbols('s')
delta = sm.symbols('delta')
n = sm.symbols('n')
g = sm.symbols('g')

The steady state solution $z^*$ is found. However, it seems SymPy is having trouble solving for $z^*$ in (1). 

In [ ]:
ss = sm.Eq(z,(((s+(1-delta)*z)**(1-alpha) * z**(alpha))/(((1+g)*(1+n))**(beta)))) # define equation (1)
display(ss)
# attempt to solve using sm.solve
try:
    z_ss = sm.solve(ss,z)[0]
except:
    print('SymPy could not find a solution.')

It is speculated that SymPy has issues with too many exponents when variables/parameters are not explicitly bounded. (1) is simplied a bit to help SymPy:
$$
\begin{align*}
z^* &=\frac{1}{[(1+g)(1+n)]^{\beta}}[s + (1-\delta)z^*]^{1-\alpha}\left(z^*\right)^{\alpha}\\
1 &=\frac{1}{[(1+g)(1+n)]^{\beta}}[s + (1-\delta)z^*]^{1-\alpha}\left(z^*\right)^{\alpha-1}\\
1 &=\frac{1}{[(1+g)(1+n)]^{\beta}}[s + (1-\delta)z^*]^{1-\alpha}\left(z^*\right)^{-(1-\alpha)}\\
1 &=\frac{1}{[(1+g)(1+n)]^{\beta}}\left[\frac{s}{z^*} + (1-\delta)\right]^{1-\alpha}\\
1 &=\left(\frac{1}{[(1+g)(1+n)]^{\beta}}\right)^{\frac{1}{1-\alpha}}\left[\frac{s}{z^*} + (1-\delta)\right]\\
\end{align*}
$$


Using this, SymPy now finds a solution for $z^*$:

In [ ]:
ss = sm.Eq(1,((1/((1+g)*(1+n))**(beta))**(1/(1-alpha))*(s/z+(1-delta)))) # define the simplified steady state equation above
z_ss = sm.solve(ss,z)[0] # solve for z^*

# display the solution
display(z_ss)

The solution is turned into a Python function using `sm.lambdify`.

In [ ]:
ss_func=sm.lambdify(args = (s,g,n,delta,alpha,beta), expr = z_ss) # turn the symbolic solution into a function

# print type of ss_func
print(f'ss_func is of the type: {type(ss_func)}.')

## 3. <a id='toc3_'></a>[Numerical solution](#toc0_)

Now, the steady value of $z$ is found numerically using `optimize.root_scalar` and specifically the 'brentq' method, which is very efficient for finding the root of a function in a bracketing interval.

In [ ]:
# set plausible values of the parameters
alpha = 0.3
beta = 0.6
kappa = 1 - alpha - beta
s = 0.2
delta = 0.05
n = 0.015
g = 0.02

# find steady state value of z using numerical optimization
obj = lambda z_star: z_star - (((s+(1-delta)*z_star)**(1-alpha) * z_star**(alpha))/((1+g)*(1+n))**(beta))
result = optimize.root_scalar(obj,bracket=[0.1,100],method='brentq', options={'disp':True})
z_star = result.root # store the solution in a variable
print(result) 
print(f'\n The steady state for z is z*={z_star:.3f}')  

The optimization converged without problems (and does so for all allowed parameter values).

Next, the model is simulated:

In [ ]:
# 1. Define the production function 
def production(K, L, A, X, alpha=alpha, beta=beta, kappa=kappa):
    return K**alpha * (A * L)**beta * X**kappa

# 2. Define the initial values of the variables
X = 1
K0 = 1
L0 = 1
A0 = 1
Y0 = production(K0, L0, A0, X)
z0 = K0 / Y0

# 3. Define a function to update the variables in each period
def update(K, L, A, X, alpha, beta, kappa, s, delta, n, g):
    Y = production(K, L, A, X, alpha, beta, kappa)
    K_new = s * Y + (1 - delta) * K
    L_new = (1 + n) * L
    A_new = (1 + g) * A
    Y_new = production(K_new, L_new, A_new, X, alpha, beta, kappa)
    return Y_new, K_new, L_new, A_new

# 4. Run a loop to simulate the model
T = 200 # iterations
K = np.empty(T) # initialize empty list
L = np.empty(T)
A = np.empty(T)
Y = np.empty(T)
z = np.empty(T)

K[0] = K0 # set first element of list to initial value
L[0] = L0
A[0] = A0
Y[0] = Y0
z[0] = z0

for t in range(1, T):
    Y[t], K[t], L[t], A[t] = update(K[t-1], L[t-1], A[t-1], X, alpha, beta, kappa, s, delta, n, g)
    z[t] = K[t] / Y[t]

In [ ]:
# print first few iterations of simulation
for i in [0, 1, 2, 3, 4]:
    print(f'{i}  {Y[i]:.3f}  {K[i]:.3f}  {L[i]:.3f}  {A[i]:.3f}')

Next, the simulation results for the model variables are plotted.

In [ ]:
# plot the simulation results
# create subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# plot labor force in the first plot
axs[0, 0].plot(range(T), L)
axs[0, 0].set_title('$L_t$')
axs[0, 0].set_xlabel('$t$')

# plot technology in the second plot
axs[0, 1].plot(range(T), A)
axs[0, 1].set_title('$A_t$')
axs[0, 1].set_xlabel('$t$')

# plot capital and output in the third plot
axs[1, 0].plot(range(T), K, label='$K_t$')
axs[1, 0].plot(range(T), Y, label='$Y_t$')
axs[1, 0].set_title('$K_t$ and $Y_t$')
axs[1, 0].set_xlabel('$t$')
axs[1, 0].legend()

# plot capital per worker and output per worker in the fourth plot
axs[1, 1].plot(range(T), np.log(K/L), label='$\ln(K_t/L_t)$')
axs[1, 1].plot(range(T), np.log(Y/L), label='$\ln(Y_t/L_t)$')
axs[1, 1].set_title('$\ln(K_t/L_t)$ and $\ln(Y_t/L_t)$')
axs[1, 1].set_xlabel('$t$')
axs[1, 1].legend()

# adjust spacing between subplots
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)

# show plots
plt.show()

The plots show that the growth in capital per worker and output per converges to a steady state growth rate (the graphs in log-levels converge asymptotically to a line with slope equal to the steady state growth rate). This steady growth rate can easily be derived analytically, but that is not within the scope of this project. Similarly, it can be shown that the inclusion of land as a fixed resource in the production function implies a growth drag, since the amount of land per worker declines as the population increases.

Next, the capital-output ratio $z_t$ and its steady state value $z^*$ is plotted:

In [ ]:
# plot capital-output ratio z
plt.plot(range(T), z, label='$z_t$',  c='green') # plot z_t
plt.plot(range(T), np.repeat(z_star,T), label='$z^*$', linestyle='dashed', c='green') # plot z^* as dashed line
plt.title('$z_t$') # plot title
plt.xlabel('$t$') # x-axis label
plt.legend() # show legend

# show plot
plt.show()

The plot shows that z converges asymptotically towards its steady state level $z^*$.

## 4. <a id='toc4_'></a>[Further analysis](#toc0_)

In this section, the value of the savings rate $s$ is varied. The effect on the model is illustrated visually.

The code plots the simulation of $z$ for each value of $s$ using a loop, making the graph color darker for each iteration.

In [ ]:
# list of s values
s_list = [0.2, 0.4, 0.6, 0.8]

# create the figure
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

# make plot loop
for i in s_list:

    # update s
    s = i

    # compute and print steady state
    steady = ss_func(s,g,n,delta,alpha,beta)
    print(f'For s={i}, the steady state value of z is z*={steady:.3f}')

    K[0] = K0 # set first element of list to initial value
    L[0] = L0
    A[0] = A0
    Y[0] = Y0
    z[0] = z0

    for t in range(1, T):
        Y[t], K[t], L[t], A[t] = update(K[t-1], L[t-1], A[t-1], X, alpha, beta, kappa, s, delta, n, g)
        z[t] = K[t] / Y[t]

    ax.plot(range(T),z,label='$s=$'+str(s),c=(0.,1-i,0.)) # c is the color, the argument is the color in rgb format (in percentages of 255)
    
ax.legend(loc='upper left',prop={'size':9}); # Activate legend (uses the labels in ax.plot())
ax.set_xlabel('$t$')
plt.title('$z_t$ for varying values of $s$') # plot title

# show plot
plt.show()

The plot shows that as the savings rate $s$ increases, the steady state value of the capital-output ratio $z^*$ increases. This makes sense intuitively: as the savings rate increases, a larger share of output is saved rather than consumed, meaning that capital accumulation is larger, also in steady state.

## 5. <a id='toc5_'></a>[Model extension](#toc0_)
The model is now extended to include an exhaustible natural resource, such as oil.

The production function is now 
$$
Y_t = K_t^{\alpha}(A_tL_t)^{\beta}X^{\kappa}E_t^{\varepsilon},
$$
where $\alpha, \beta, \kappa, \varepsilon > 0$ and $\alpha + \beta + \kappa + \varepsilon = 1$. 

The stock of oil $R_t$ is depleted when used in production $E_t$, with depletion defined as 
$$
R_{t+1} = R_t - E_t.
$$
It is assumed than in each period, a constant fraction $s_E$ (the extraction rate) of the remaining stock of oil is used in production,
$$
E_t = s_E R_t,
$$
where $0<s_E<1$.

The rest of the model equations regarding population, technology and capital accumulation are defined as before. 

It can be shown that the steady state value of $z_t$ in this model is
$$
z^* = \frac{1}{((1+n)(1+g))^{\frac{\beta}{\beta+\kappa+\varepsilon}}(1-s_E)^{\frac{\varepsilon}{\beta+\kappa+\varepsilon}}-(1-\delta)}s
$$
The model is now solved numerically:

In [ ]:
# set plausible values of the parameters (except for s_E, which is set so that illustrations are interesting)
alpha = 0.3
beta = 0.5
kappa = 0.1
epsilon = 1 - alpha - beta - kappa
s = 0.2
s_E = 0.05

# find steady state value of z using numerical optimization
obj = lambda z_star: z_star-(1/(((1+n)*(1+g))**(beta/(beta+kappa+epsilon))*(1-s_E)**(epsilon/(beta+kappa+epsilon))-(1-delta))*s)
result = optimize.root_scalar(obj,bracket=[0.01,200],method='brentq', options={'disp':True})
z_star = result.root # store the solution in a variable
print(result) 
print(f'\n The steady state for z is z*={z_star:.3f}')  

The optimization converged. However, too high values of $s_E$ can cause problems for the convergence.

Next, the model is simulated as before.

In [ ]:
# 1. Define the production function 
def production(K, L, A, X, E, alpha=alpha, beta=beta, kappa=kappa, epsilon=epsilon):
    return K**alpha * (A * L)**beta * X**kappa * E**epsilon

# 2. Define the initial values of the variables
X = 1
R0 = 1
E0 = R0*s_E
Y0 = production(K0, L0, A0, X, E0)
z0 = K0 / Y0

# 3. Define update function
def update(K, L, A, X, E, alpha, beta, kappa, epsilon, s, delta, n, g):
    Y = production(K, L, A, X, E, alpha, beta, kappa, epsilon)
    R = E / s_E
    K_new = s * Y + (1 - delta) * K
    L_new = (1 + n) * L
    A_new = (1 + g) * A
    R_new = R - E
    E_new = R_new * s_E
    Y_new = production(K_new, L_new, A_new, X, E_new, alpha, beta, kappa, epsilon)
    return Y_new, K_new, L_new, A_new, E_new, R_new

# 4. Run a loop to simulate the model
T = 200 # iterations
K = np.empty(T) # initialize empty list
L = np.empty(T)
A = np.empty(T)
Y = np.empty(T)
R = np.empty(T)
E = np.empty(T)
z = np.empty(T)

K[0] = K0 # set first element of list to initial value
L[0] = L0
A[0] = A0
Y[0] = Y0
R[0] = R0
E[0] = E0
z[0] = z0

for t in range(1, T):
    Y[t], K[t], L[t], A[t], E[t], R[t] = update(K[t-1], L[t-1], A[t-1], X, E[t-1], alpha, beta, kappa, epsilon, s, delta, n, g)
    z[t] = K[t] / Y[t]

In [ ]:
# plot the simulation results
# create subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# plot stock of oil in the first plot
axs[0, 0].plot(range(T), R, c='grey')
axs[0, 0].set_title('$R_t$')
axs[0, 0].set_xlabel('$t$')

# plot usage of oil in production in the second plot
axs[0, 1].plot(range(T), E, c='black')
axs[0, 1].set_title('$E_t$')
axs[0, 1].set_xlabel('$t$')

# plot capital per worker and output per worker in the third plot
axs[1, 0].plot(range(T), np.log(K/L), label='$\ln(K/L)$')
axs[1, 0].plot(range(T), np.log(Y/L), label='$\ln(Y/L)$')
axs[1, 0].set_title('$\ln(K/L)$ and $\ln(Y/L)$')
axs[1, 0].set_xlabel('$t$')
axs[1, 0].legend()

# plot capital-output ratio z in fourth plot
plt.plot(range(T), z, label='$z_t$',  c='green') # plot z_t
plt.plot(range(T), np.repeat(z_star,T), label='$z^*$', linestyle='dashed', c='green') # plot z^* as dashed line
plt.title('$z_t$') # plot title
plt.xlabel('$t$') # x-axis label
plt.legend() # show legend

# adjust spacing between subplots
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)

# show plots
plt.show()

The stock of oil $R_t$ depletes exponentially, and so does the amount of oil used in production $E_t$. Otherwise, as shown by the plots, conclusions are similar to before. Output per worker converges on a steady state growth path and $z_t$ converges to a steady state level $z^*$. It can be shown that the inclusion of an exhaustible resource like oil in addition to land adds further drag on the growth rate.


**More ideas for an extended project:** In the first model with only land, instead of varying the paramater $s$, one could vary the parameter $\kappa$, which controls land's income share. In the second model, it would be interesting to vary the depletion rate $s_E$. Interestingly, as $s_E$ increases, the steady value of $z_t$ increases, but the steady state growth rate in output per worker **decreases**! This is so, because in the short-run, a higher $s_E$ increases productivity, but in the long run, it falls!

One could further extend this project by including the firms' profit maximization problem, which is a part of the original model. Other ideas could be to make explicit microfoundations for the households' choices, include human capital, use a CES production function instead of a Cobb-Douglas or perhaps look at an endogenous growth model. Using a CES production function would have implications on the elasticity of substitution between production factors.

## 6. <a id='toc6_'></a>[Conclusion](#toc0_)

When the fixed resource land is included in the Solow model, there is still growth in output per worker and capital per worker, albeit with a growth drag. In this specification, the model is solved using the capital-output ratio, which converges to a steady state level. Including a scarce, depletable natural resource such as oil puts even more drag on growth. However, in both cases, the Cobb-Douglas production function and technological growth allows for long-run growth.